# 营销活动优化

## 目标和前提条件

几乎所有行业的公司都在寻求优化其营销活动。在本Jupyter笔记本中，我们将探讨银行和金融服务行业常见的营销活动优化问题，该问题涉及确定向个别客户提供哪些产品，以在满足各种业务约束的同时实现预期总利润最大化。您将学习如何制定问题的数学优化模型（使用机器学习预测响应模型作为参数），并使用Gurobi优化器求解。

这个建模示例属于初级水平，我们假设您了解Python，并且对构建数学优化模型有一定了解。读者还应参考Gurobi Python API的[文档](https://www.gurobi.com/resources/?category-filter=documentation)。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 动机

银行和金融服务业营销的主要目标是"在正确的时间向正确的客户提供正确的产品"。然而，实际实现这个目标是一个复杂且具有挑战性的任务。尤其困难的是，公司拥有多种产品并在复杂的业务约束下运营。在满足业务约束的同时选择向哪些客户提供哪些产品以最大化营销投资回报率是极其复杂的。

考虑一家主要银行，该银行已经做出了成为以客户为中心的机构的明确努力，而不是垂直产品驱动的公司。该银行的目标是"通过为客户的特定需求提供相关解决方案，帮助客户在财务上变得更好"。这个目标的直接结果是营销活动是多产品活动，而不是单一产品活动。这将数据科学和活动定位过程从单个响应模型的相对简单应用转变为涉及选择向哪个客户通过哪个渠道提供哪个产品的更复杂过程。

该银行的营销团队习惯于直接应用业务规则来锁定客户。例如，他们仅根据客户的产品缺口或营销人员的商业直觉来锁定客户。该银行的营销人员还应用了RFM类型的分析，其中使用一般的近期性、频率和货币度量以及产品缺口来为特定优惠锁定客户。

营销团队当前的方法（广泛使用）依赖于预测响应模型来锁定客户进行优惠。这些模型估计客户对特定优惠做出响应的概率，可以显著提高产品优惠的响应率。然而，当公司有多个产品要推广并且需要考虑其他业务约束时，仅仅知道客户对特定优惠的响应概率是不够的。

一般来说，营销团队还面临着了解向客户提供哪些产品的问题，而不仅仅是向哪些客户提供产品。在实践中，使用了许多临时规则：

* 基于响应率或估计预期盈利能力措施的优先级规则

* 用于确定可营销产品优先级的业务规则

* 用于为特定活动选择客户的产品响应模型

一种容易实施但可能无法产生最佳客户联系计划的方法是依靠预期优惠盈利能力的衡量来选择向客户提供哪些产品。然而，这种方法的一个缺点是无法有效处理客户联系计划的复杂约束。

为解决这个营销活动优化问题，M. D. Cohen [1]提出了一个使用丰业银行数据的MIP方法。考虑的营销活动优化问题包括十一个独特的优惠：五个投资、三个贷款和三个日常银行业务优惠。投资优惠包括保证投资存单(GIC)、共同基金、注册教育储蓄计划(RESP)和两个独特的折扣经纪优惠。贷款优惠包括一个抵押贷款和两个信用卡优惠。日常银行业务优惠包括两个丰业在线银行服务优惠之一和一个存款账户获取。这里使用的术语"活动"意味着一个由十一个不同优惠组成的大型主动客户联系活动；它可以被认为是在大致相同时间向非重叠客户群提供的十一个单一产品活动。该活动的潜在目标市场包括约250万客户。


在本Jupyter笔记本中，我们将使用这种MIP方法来解决银行的营销活动优化问题。值得注意的是，几乎任何行业的公司都可以使用这种方法来优化其营销活动，同时考虑其业务约束。


## 问题描述
银行的营销团队需要确定以何种方式向每个客户提供产品，以最大化营销活动投资回报，同时考虑以下约束：

* 活动可用资金限制。
* 在活动中可以提供的最少产品数量的限制。
* 必须满足的活动投资回报率门槛。

## 解决方案方法

数学规划是一种声明性方法，模型制定者通过制定数学优化模型来捕捉复杂决策问题的关键方面。Gurobi优化器使用最先进的数学和计算机科学来求解这些模型。

数学优化模型有五个组成部分，即：

* 集合和索引。
* 参数。
* 决策变量。
* 目标函数。
* 约束条件。

我们现在为这个营销活动优化问题提出一种MIP方法。

Cohen [1]提出的MIP解决方案方法是对传统的逐步选择最大预期值客户方法的改进，因为它从银行的角度产生全局最优解，并允许在客户和业务单元之间有效实施业务约束。该方法考虑了有限的资源和其他业务约束。

我们假设客户/优惠预期增量利润、成本和业务约束的估计值作为营销活动优化方法的输入。优化阶段独立于这些输入的构建。

MIP方法涉及战术问题和操作问题。对于战术问题，我们根据个体预期利润参数对客户进行聚类。个体预期利润的估计可以通过数据科学技术（如预测响应模型）来确定。关键思想是对个体预期利润进行聚类，然后将聚类中心视为单个聚类中所有个体客户数据的代表。这种聚合使得问题可以被表述为线性规划问题，因此模型识别每个聚类中每个产品优惠的比例，以最大化营销活动投资回报，同时考虑业务约束。通常，每个聚类中的客户数量将达到数十万，这是战术问题的主要决策变量，因此这些变量可以被视为连续变量；因此，线性规划方法是合理的。

操作问题可以表述为MIP模型，其中个体预期利润的估计值和战术模型的输出可以用作输入，以便在每个聚类中为个体客户分配产品优惠，从而最大化总营销活动投资回报。

## 战术模型公式

### 集合和索引
$k \in K$: 聚类的索引和集合。

$j \in J$: 产品的索引和集合。

### 参数
$\pi_{k,j}$: 向聚类$k \in K$的平均客户提供产品$j \in J$的预期利润。

$\nu_{k,j}$: 向聚类$k \in K$的平均客户提供产品$j \in J$的平均可变成本。
  
$N_{k}$: 聚类$k \in K$中的客户数量。

$Q_{j}$: 产品$j \in J$的最少优惠数量。

$R$: 企业门槛率。此门槛率用于营销活动的ROI计算。

$B$: 营销活动预算。

$M$: 大M惩罚。此惩罚与满足其他业务约束所需的预算修正相关。
 

### 决策变量
$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$z \geq 0$: 为了使活动可行而增加的预算。

### 目标函数
- **总利润**。最大化营销活动的总预期利润，并严重惩罚任何预算修正。

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}

### 约束条件

- **优惠数量**。每个聚类的产品优惠数量最多受限于聚类中的客户数量。

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

- **预算**。营销活动预算约束强制要求活动的总成本应小于预算活动。为了确保模型的可行性，有可能增加预算，因为所有产品的最少优惠数量可能需要增加预算。

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}

- **优惠限制**。每种产品的最少优惠数量。

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

- **ROI**。最小ROI约束确保总利润与成本的比率至少为企业门槛率加一。

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}


## 操作模型公式

一旦找到了战术模型的最优值$y_{k,j}$，对于所有$j \in J$和$k \in K$，我们应该确定聚类$k$中的哪些个体客户应该获得产品优惠。假设对于给定的聚类$k \in K$，产品$j_1$和$j_2$的优惠分配是正的，即$y_{k,j_1} > 0$和$y_{k,j_2} > 0$。然后，聚类$k$中的$y_{k,j_1}$和$y_{k,j_2}$客户必须分别获得产品$j_1$和$j_2$的优惠。最优的方法是使用个体客户的预期利润而不是聚类的预期利润来解决分配问题。

我们现在提供操作问题的公式。

### 集合和索引
$i \in I^{k}$: 聚类$k \in K$中的客户索引和集合。

$j \in J^{k}$: 聚类$k \in K$中提供给客户的产品索引和子集，其中$J^{k} = \{ j \in J: y_{k,j} > 0 \}$。

### 参数

$r_{k,i,j}$: 客户$i \in I^{k}$从产品$j \in J^{k}$的优惠中获得的预期个体利润。

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: 聚类$k$中将获得产品$j \in J^{k}$优惠的客户数量。

### 决策变量
$x_{k,i,j} \in \{0,1 \}$: 如果产品$j \in J^{k}$提供给客户$i \in I^{k}$，则该变量等于1，否则为0。



### 目标函数
- **总利润**。最大化总个体利润。

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}


### 约束条件

- **产品优惠**。将产品优惠分配给每个聚类的客户。

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}


- **优惠限制**。每个聚类的客户最多可以获得一个产品优惠。

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **二进制约束**。要么向聚类$k$的客户提供产品优惠，要么不提供。

\begin{equation}
x_{k,i,j} \in \{0,1 \} \quad \forall i \in I^{k},  j \in J^{k}, k \in K
\tag{3}
\end{equation}


## 问题实例

我们考虑两个产品、十个客户和两个客户聚类。企业门槛率为20%。

### 战术问题数据

下表定义了向每个聚类的平均客户提供产品时的预期利润。

| <i></i> | 产品1 | 产品2 |
| --- | --- |  --- |
| 聚类1 | $\$2,000$ | $\$1,000$ |
| 聚类2 | $\$3,000$ | $\$2,000$ |

向每个聚类的平均客户提供产品的预期成本由下表确定。

| <i></i> | 产品1 | 产品2 |
| --- | --- |  --- |
| 聚类1 | $\$200$ | $\$100$ |
| 聚类2 | $\$300$ | $\$200$ |

营销活动的可用预算为$\$200$。

每个聚类的客户数量如下表所示。

| <i></i> | 客户数量 | 
| --- | --- |
| 聚类1 | 5 |
| 聚类2 | 5 | 

每种产品的最少优惠数量如下表所示，

| <i></i> | 最少优惠数量 | 
| --- | --- |
| 产品1 | 2 |
| 产品2 | 2 | 

### 操作问题数据

下表显示了向每个聚类的客户提供产品时的预期利润。

| <i></i> | 产品1 | 产品2 |
| --- | --- |  --- |
| 聚类1，客户1 | $\$2,050$ | $\$1,050$ |
| 聚类1，客户2 | $\$1,950$ | $\$950$ |
| 聚类1，客户3 | $\$2,000$ | $\$1,000$ |
| 聚类1，客户4 | $\$2,100$ | $\$1,100$ |
| 聚类1，客户5 | $\$1,900$ | $\$900$ |
| 聚类2，客户6 | $\$3,000$ | $\$2,000$ |
| 聚类2，客户7 | $\$2,900$ | $\$1,900$ |
| 聚类2，客户8 | $\$3,050$ | $\$2,050$ |
| 聚类2，客户9 | $\$3,100$ | $\$2,100$ |
| 聚类2，客户10 | $\$2,950$ | $\$1,950$ |

下表显示了向每个聚类的客户提供产品的成本。

| <i></i> | 产品1 | 产品2 |
| --- | --- |  --- |
| 聚类1，客户1 | $\$205$ | $\$105$ |
| 聚类1，客户2 | $\$195$ | $\$95$ |
| 聚类1，客户3 | $\$200$ | $\$100$ |
| 聚类1，客户4 | $\$210$ | $\$110$ |
| 聚类1，客户5 | $\$190$ | $\$90$ |
| 聚类2，客户6 | $\$300$ | $\$200$ |
| 聚类2，客户7 | $\$290$ | $\$190$ |
| 聚类2，客户8 | $\$305$ | $\$205$ |
| 聚类2，客户9 | $\$310$ | $\$210$ |
| 聚类2，客户10 | $\$295$ | $\$195$ |

## Python实现

我们现在导入Gurobi Python模块。然后，我们使用给定的数据初始化数据结构。

In [ ]:
# %pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 使用Gurobi v9.0.0和Python 3.7.0测试

### 集合

products = ['p1', 'p2']
clusters = ['k1', 'k2']

### 预期利润

下表显示了向每个聚类的客户提供产品时的预期利润。

| <i></i> | 产品1 | 产品2 |
| --- | --- |  --- |
| 聚类1 | $\$2,000$ | $\$1,000$ |
| 聚类2 | $\$3,000$ | $\$2,000$ |

In [ ]:
### 参数

# 预期利润
cp, expected_profit = gp.multidict({
    ('k1', 'p1'): 2000,
    ('k1', 'p2'): 1000,
    ('k2', 'p1'): 3000,
    ('k2', 'p2'): 2000
})


### 预期成本

下表显示了向每个聚类的客户提供产品的预期成本。

| <i></i> | 产品1 | 产品2 |
| --- | --- |  --- |
| 聚类1 | $\$200$ | $\$100$ |
| 聚类2 | $\$300$ | $\$200$ |

In [ ]:
# 预期成本

cp, expected_cost = gp.multidict({
    ('k1', 'p1'): 200,
    ('k1', 'p2'): 100,
    ('k2', 'p1'): 300,
    ('k2', 'p2'): 200
})

### 客户数量
下表显示了每个聚类的客户数量。

| <i></i> | 客户数量 | 
| --- | --- |
| 聚类1 | 5 |
| 聚类2 | 5 | 

In [ ]:
# 每个聚类的客户数量

clusters, number_customers = gp.multidict({
    ('k1'): 5,
    ('k2'): 5
})

### 最少优惠数量

下表显示了每种产品的最少优惠数量。

| <i></i> | 最少优惠数量 | 
| --- | --- |
| 产品1 | 2 |
| 产品2 | 2 | 

In [ ]:
# 每个产品的最小优惠数量

products, min_offers = gp.multidict({
    ('p1'): 2,
    ('p2'): 2
})

### 标量

企业门槛率为20%（$R = 0.20$）。

营销活动的可用预算为$\$200$。


In [ ]:
# 标量

R = 0.20

#紧张预算
budget = 200 

## 战术模型公式
 

### 决策变量
$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$z \geq 0$: 为了使活动可行而增加的预算。

In [ ]:
# 声明和初始化模型
mt = gp.Model('Tactical')

### 决策变量

# 分配产品优惠给聚类中的客户

y = mt.addVars(cp, name="allocate")

# 预算修正

z = mt.addVar(name="budget_correction")

Using license file c:\gurobi\gurobi.lic


### 约束条件

- **优惠数量**。每个聚类的产品优惠数量最多受限于聚类中的客户数量。

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

其中

$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$N_{k}$: 聚类$k \in K$中的客户数量。

In [ ]:
### 约束条件

# 每个聚类的优惠数量约束

maxOffers_cons = mt.addConstrs((y.sum(k,'*') <= number_customers[k]  for k in clusters), name='maxOffers')


### 约束条件

- **预算**。营销活动预算约束强制要求活动的总成本应小于预算活动。为了确保模型的可行性，有可能增加预算，因为所有产品的最少优惠数量可能需要增加预算。

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}

其中

$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$z \geq 0$: 为了使活动可行而增加的预算。

$\nu_{k,j}$: 向聚类$k \in K$的平均客户提供产品$j \in J$的平均可变成本。

$B$: 营销活动预算。

In [ ]:
# 预算约束

budget_con = mt.addConstr((y.prod(expected_cost) - z <= budget), name='budget')


### 约束条件

- **优惠限制**。每种产品的最少优惠数量。

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

其中

$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$Q_{j}$: 产品$j \in J$的最少优惠数量。

In [ ]:
# 每个产品的最小优惠数量约束

minOffers_cons = mt.addConstrs( (y.sum('*',j) >= min_offers[j] for j in products), name='min_offers')


### 约束条件


- **ROI**。最小ROI约束确保总利润与成本的比率至少为企业门槛率加一。

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}

其中

$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$\pi_{k,j}$: 向聚类$k \in K$的平均客户提供产品$j \in J$的预期利润。

$\nu_{k,j}$: 向聚类$k \in K$的平均客户提供产品$j \in J$的平均可变成本。

$R$: 企业门槛率。

In [ ]:
# 确保最小投资回报率的约束

ROI_con = mt.addConstr((y.prod(expected_profit) - (1 + R)*y.prod(expected_cost) >= 0), name='ROI')

### 目标函数
- **总利润**。最大化营销活动的总预期利润，并严重惩罚任何预算修正。

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}

其中

$y_{k,j} \geq 0$: 向聚类$k \in K$中的客户提供产品$j \in J$的数量。

$z \geq 0$: 为了使活动可行而增加的预算。

$\pi_{k,j}$: 向聚类$k \in K$的平均客户提供产品$j \in J$的预期利润。

**注意：** $M$的值应高于任何预期利润，以确保仅在模型不可行时才增加预算。

In [ ]:
### 目标函数

# 最大化总预期利润

M = 10000

mt.setObjective(y.prod(expected_profit) -M*z, GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

mt.write('tactical.lp')

In [ ]:
# 运行优化引擎

mt.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 5 columns and 17 nonzeros
Model fingerprint: 0x1eac2f22
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+03, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+04   8.787500e+01   0.000000e+00      0s
       4   -3.9940000e+06   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective -3.994000000e+06


In [ ]:
### 输出报告

# 优化产品优惠分配到聚类

total_expected_profit = 0
total_expected_cost = 0

print("\nOptimal allocation of product offers to clusters.")
print("___________________________________________________")
for k,p in cp:
    if y[k,p].x > 1e-6:
        #print(y[k,p].varName, y[k,p].x)
        print(f"The number of customers in cluster {k} that gets an offer of product {p} is: {y[k,p].x}")
        total_expected_profit += expected_profit[k,p]*y[k,p].x
        total_expected_cost += expected_cost[k,p]*y[k,p].x

increased_budget = '${:,.2f}'.format(z.x)
print(f"\nThe increase correction in the campaign budget is {increased_budget}.")

# 财务报告

optimal_ROI = round(100*total_expected_profit/total_expected_cost,2)
min_ROI = round(100*(1+R),2)

money_expected_profit = '${:,.2f}'.format(total_expected_profit)
money_expected_cost = '${:,.2f}'.format(total_expected_cost)
money_budget = '${:,.2f}'.format(budget)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total expected profit is {money_expected_profit}.")
print(f"Optimal total expected cost is {money_expected_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {optimal_ROI}% with a minimum ROI of  {min_ROI}%.")


Optimal allocation of product offers to clusters.
___________________________________________________
The number of customers in cluster k1 that gets an offer of product p1 is: 2.0
The number of customers in cluster k1 that gets an offer of product p2 is: 2.0

The increase correction in the campaign budget is $400.00.

Financial reports.
___________________________________________________
Optimal total expected profit is $6,000.00.
Optimal total expected cost is $600.00 with a budget of $200.00 and an extra amount of $400.00.
Optimal ROI is 1000.0% with a minimum ROI of  120.0%.


## 分析

分配产品到聚类的成本需要增加$\$400$的预算。总预期利润为$\$6,000$。总预期成本为$\$600$，等于原始预算$\$200$加上$\$400$的增加。预期ROI为1,000%，远高于所需的最低ROI。

## 操作模型公式

### 客户预期利润

In [ ]:
### 集合

customers = ['c1', 'c2','c3','c4','c5','c6','c7','c8','c9','c10']

### 参数

# 每个聚类中每个客户的产品优惠预期利润
ccp, customer_profit = gp.multidict({
    ('k1', 'c1', 'p1'): 2050,
    ('k1', 'c1', 'p2'): 1050,
    ('k1', 'c2', 'p1'): 1950,
    ('k1', 'c2', 'p2'): 950,
    ('k1', 'c3', 'p1'): 2000,
    ('k1', 'c3', 'p2'): 1000,
    ('k1', 'c4', 'p1'): 2100,
    ('k1', 'c4', 'p2'): 1100,
    ('k1', 'c5', 'p1'): 1900,
    ('k1', 'c5', 'p2'): 900,
    ('k2', 'c6', 'p1'): 3000,
    ('k2', 'c6', 'p2'): 2000,
    ('k2', 'c7', 'p1'): 2900,
    ('k2', 'c7', 'p2'): 1900,
    ('k2', 'c8', 'p1'): 3050,
    ('k2', 'c8','p2'): 2050,
    ('k2', 'c9', 'p1'): 3100,
    ('k2', 'c9', 'p2'): 3100,
    ('k2', 'c10', 'p1'): 2950,
    ('k2', 'c10', 'p2'): 2950   
})

### 客户提供成本

In [ ]:
# 每个聚类中每个客户的产品优惠成本

ccp, customer_cost = gp.multidict({
    ('k1', 'c1', 'p1'): 205,
    ('k1', 'c1', 'p2'): 105,
    ('k1', 'c2', 'p1'): 195,
    ('k1', 'c2', 'p2'): 95,
    ('k1', 'c3', 'p1'): 200,
    ('k1', 'c3', 'p2'): 100,
    ('k1', 'c4', 'p1'): 210,
    ('k1', 'c4', 'p2'): 110,
    ('k1', 'c5', 'p1'): 190,
    ('k1', 'c5', 'p2'): 90,
    ('k2', 'c6', 'p1'): 300,
    ('k2', 'c6', 'p2'): 200,
    ('k2', 'c7', 'p1'): 290,
    ('k2', 'c7', 'p2'): 190,
    ('k2', 'c8', 'p1'): 305,
    ('k2', 'c8','p2'): 205,
    ('k2', 'c9', 'p1'): 310,
    ('k2', 'c9', 'p2'): 310,
    ('k2', 'c10', 'p1'): 295,
    ('k2', 'c10', 'p2'): 295   
})

## 操作模型公式


### 决策变量
$x_{k,i,j} \in \{0,1 \}$: 如果产品$j \in J^{k}$提供给客户$i \in I^{k}$，则该变量等于1，否则为0。

In [ ]:
# 声明和初始化模型
mo = gp.Model('Operational')

### 决策变量

x = mo.addVars(ccp, vtype=GRB.BINARY, name="assign")

### 约束条件

- **产品优惠**。将产品优惠分配给每个聚类的客户。

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

其中

$x_{k,i,j} \in \{0,1 \}$: 如果产品$j \in J^{k}$提供给客户$i \in I^{k}$，则该变量等于1，否则为0。

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: 聚类$k$中将获得产品$j \in J^{k}$优惠的客户数量。




In [ ]:
# 产品优惠约束

productOffers = {}

for k in clusters:
    for j in products:
            productOffers[k,j] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffers_' + str(k) + ',' + str(j) )


### 约束条件


- **优惠限制**。每个聚类的客户最多可以获得一个产品优惠。

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

其中

$x_{k,i,j} \in \{0,1 \}$: 如果产品$j \in J^{k}$提供给客户$i \in I^{k}$，则该变量等于1，否则为0。

In [ ]:
# 每个聚类中每个客户的优惠数量限制

ki = [('k1', 'c1'), 
      ('k1', 'c2'), 
      ('k1', 'c3'),
      ('k1', 'c4'), 
      ('k1', 'c5'), 
      ('k2', 'c6'), 
      ('k2', 'c7'), 
      ('k2', 'c8'), 
      ('k2', 'c9'), 
      ('k2', 'c10')]

customerOffers = {}

for k,i in ki:
    customerOffers[k,i] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffers_' + str(k) + ',' + str(i) )

### 目标函数

- **总利润**。最大化总个体预期利润。

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

其中

$x_{k,i,j} \in \{0,1 \}$: 如果产品$j \in J^{k}$提供给客户$i \in I^{k}$，则该变量等于1，否则为0。

$r_{k,i,j}$: 客户$i \in I^{k}$从产品$j \in J^{k}$的优惠中获得的预期个体利润。



In [ ]:
### 目标函数

# 最大化总利润

mo.setObjective(x.prod(customer_profit), GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

mo.write('operational.lp')

# 运行优化引擎

mo.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 14 rows, 20 columns and 40 nonzeros
Model fingerprint: 0xe1e9d99f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6050.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.100000e+03, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6100.0000000 6100.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.02 seconds
Thread cou

In [ ]:
### 输出报告

# 优化产品优惠分配给客户

total_customer_profit = 0
total_customer_cost = 0

kvalue = None
first = True
num_assignments = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalue:
        prevk = kvalue
        kvalue = k
        if not first:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevk} is {num_assignments}")
            print("___________________________________________________")
            num_assignments = 0
        if first:
            first = False
    if x[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profit = '${:,.2f}'.format(customer_profit[k,i,j])
        cost = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profit} at a cost of {cost}")
        total_customer_profit += customer_profit[k,i,j]*x[k,i,j].x
        total_customer_cost += customer_cost[k,i,j]*x[k,i,j].x
        num_assignments += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalue} is {num_assignments}")
print("___________________________________________________\n")
        
# 财务报告

customers_ROI = round(100*total_customer_profit/total_customer_cost,2)

money_customers_profit = '${:,.2f}'.format(total_customer_profit)
money_customers_cost = '${:,.2f}'.format(total_customer_cost)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profit}.")
print(f"Optimal total customers cost is {money_customers_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROI}% with a minimum ROI of  {min_ROI}%.")
        



Optimal assignment of product offers to customers.
___________________________________________________
Customer c1 in cluster k1 gets an offer of product p2:
The expected profit is $1,050.00 at a cost of $105.00
Customer c2 in cluster k1 gets an offer of product p2:
The expected profit is $950.00 at a cost of $95.00
Customer c3 in cluster k1 gets an offer of product p1:
The expected profit is $2,000.00 at a cost of $200.00
Customer c4 in cluster k1 gets an offer of product p1:
The expected profit is $2,100.00 at a cost of $210.00
___________________________________________________
Number of assignments in cluster k1 is 4
___________________________________________________
___________________________________________________
Number of assignments in cluster k2 is 0
___________________________________________________


Financial reports.
___________________________________________________
Optimal total customers profit is $6,100.00.
Optimal total customers cost is $610.00 with a budget o

## 分析
每个客户最多获得一个产品优惠。产品p2提供给客户c1和c2，产品p1提供给客户c3和c4。产品p1和p2至少提供给两个客户——这是战术模型的约束。请注意，为了确保这些硬性业务约束，预算需要增加$\$400$。

将产品分配给客户的成本为$\$610$，略微违反了$\$600$的总可用预算。总客户利润为$\$6,100$。ROI为1,000%，远高于所需的最低ROI。

如果需要强制执行总可用预算，可以将以下约束添加到操作模型中：

- **预算**。强制执行预算约束。

\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{4}
\end{equation}

新预算为原始预算加上修正，即$B' = B + z$

## 场景1
强制执行总可用预算约束。在这种情况下，操作模型为：

### 目标函数

- **总利润**。最大化总个体预期利润。

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

### 约束条件

- **产品优惠**。将产品优惠分配给每个聚类的客户。

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

- **优惠限制**。每个聚类的客户最多可以获得一个产品优惠。

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **预算**。强制执行预算约束。

\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{3}
\end{equation}

In [ ]:
### 强制执行总预算约束的操作模型

# 声明和初始化模型
mob = gp.Model('OperationalB')

### 决策变量

xb = mob.addVars(ccp, vtype=GRB.BINARY, name="assign")

In [ ]:
# 产品优惠约束

productOffersb = {}

for k in clusters:
    for j in products:
            productOffersb[k,j] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffersb_' + str(k) + ',' + str(j) )

In [ ]:
# 每个聚类中每个客户的优惠数量限制

customerOffersb = {}

for k,i in ki:
    customerOffersb[k,i] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffersb_' + str(k) + ',' + str(i) )

In [ ]:
# 预算约束

# 新预算
new_budget = budget + z.x

totBudget = mob.addConstr(xb.prod(customer_cost) <= new_budget, name='total_budget')

In [ ]:
### 目标函数

# 最大化总利润

mob.setObjective(xb.prod(customer_profit), GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

mob.write('operationalB.lp')

# 运行优化引擎

mob.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 20 columns and 60 nonzeros
Model fingerprint: 0x96da858f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 5950.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 28 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.000000e+03, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6000.0000000 6000.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.02 seconds
Thread cou

In [ ]:
### 输出报告

# 优化产品优惠分配给客户

total_customer_profitb = 0
total_customer_costb = 0

kvalueb = None
firstb = True
num_assignmentsb = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalueb:
        prevkb = kvalueb
        kvalueb = k
        if not firstb:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevkb} is {num_assignmentsb}")
            print("___________________________________________________")
            num_assignmentsb = 0
        if firstb:
            firstb = False
    if xb[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profitb = '${:,.2f}'.format(customer_profit[k,i,j])
        costb = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profitb} at a cost of {costb}")
        total_customer_profitb += customer_profit[k,i,j]*xb[k,i,j].x
        total_customer_costb += customer_cost[k,i,j]*xb[k,i,j].x
        num_assignmentsb += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalueb} is {num_assignmentsb}")
print("___________________________________________________\n")
        
# 财务报告

customers_ROIb = round(100*total_customer_profitb/total_customer_costb,2)

money_customers_profitb = '${:,.2f}'.format(total_customer_profitb)
money_customers_costb = '${:,.2f}'.format(total_customer_costb)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profitb}.")
print(f"Optimal total customers cost is {money_customers_costb} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROIb}% with a minimum ROI of  {min_ROI}%.")


Optimal assignment of product offers to customers.
___________________________________________________
Customer c1 in cluster k1 gets an offer of product p2:
The expected profit is $1,050.00 at a cost of $105.00
Customer c2 in cluster k1 gets an offer of product p1:
The expected profit is $1,950.00 at a cost of $195.00
Customer c4 in cluster k1 gets an offer of product p1:
The expected profit is $2,100.00 at a cost of $210.00
Customer c5 in cluster k1 gets an offer of product p2:
The expected profit is $900.00 at a cost of $90.00
___________________________________________________
Number of assignments in cluster k1 is 4
___________________________________________________
___________________________________________________
Number of assignments in cluster k2 is 0
___________________________________________________


Financial reports.
___________________________________________________
Optimal total customers profit is $6,000.00.
Optimal total customers cost is $600.00 with a budget o

## 分析
每个客户最多获得一个产品优惠。产品p1和p2至少提供给两个客户。将产品分配给客户的成本为$\$600$，等于总可用预算。总客户利润为$\$6,000$。ROI为1,000%，远高于所需的最低ROI。

在这种情况下，我们强制执行总可用预算约束，并获得不同的分配。产品p1提供给客户c2和c4，产品p2提供给客户c1和c5。

## 结论

在本Jupyter笔记本中，我们讨论了营销活动对银行业的重要性。我们讨论了机器学习预测响应模型可以用来提供营销活动优化问题的输入数据。我们展示了如何将营销活动优化问题分解为战术问题和操作问题。

战术问题被表述为线性规划问题，我们聚合了由机器学习预测响应模型生成的数据。

战术问题的解决方案确定了在最大化营销活动预期利润的同时，考虑以下约束条件，向客户聚类提供哪些产品：

* 活动可用资金限制。
* 在活动中可以提供的最少产品数量的限制。
* 必须满足的活动投资回报率门槛。

操作问题被表述为MIP模型，其中客户预期利润和战术模型的输出可以用作输入，以便在每个聚类中为个体客户分配产品优惠，从而最大化总客户利润。我们考虑了操作问题的两种情况。在第一种情况下，不强制执行战术问题确定的总可用预算。这意味着该问题的最优解可能会略微违反总可用预算。在第二种情况下，我们强制执行总可用预算。


## 参考文献

[1] M. D. Cohen. *Exploiting response models—optimizing cross-sell and up-sell opportunities in banking.* Information Systems. Vol. 29. issue 4, June 2004, Pages 327-341

Copyright © 2020 Gurobi Optimization, LLC